# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

## Point
Predict: Crew 
Important feature: Cruise Line (string) -> stringindexer
Model: Linear regression

In [1]:
import findspark
findspark.init('/home/quanttraderkim/spark-2.3.1-bin-hadoop2.7')

In [4]:
from pyspark.sql import SparkSession

In [22]:
from pyspark.ml.regression import LinearRegression

In [5]:
spark = SparkSession.builder.appName('cruise').getOrCreate()

In [15]:
df = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [20]:
df.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|      NaN|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [16]:
df.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [17]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [19]:
df.select('Cruise_line').show()

+-----------+
|Cruise_line|
+-----------+
|    Azamara|
|    Azamara|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
|   Carnival|
+-----------+
only showing top 20 rows



In [105]:
df.groupBy('Ship_name').count().sort('count', ascending=False).show()

+---------+-----+
|Ship_name|count|
+---------+-----+
|   Spirit|    4|
|   Legend|    3|
|     Star|    3|
|    Crown|    2|
|    Pride|    2|
|  Freedom|    2|
|  Voyager|    2|
|     Dawn|    2|
|Navigator|    2|
| Rhapsody|    2|
|  Mariner|    2|
|  Majesty|    2|
|     Wind|    2|
|      Sun|    2|
|      Sea|    2|
|    Jewel|    2|
|Millenium|    1|
| Solstice|    1|
|   Shadow|    1|
|   Wonder|    1|
+---------+-----+
only showing top 20 rows



In [128]:
df.groupBy('Cruise_line').count().sort('count', ascending=False).show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|  Royal_Caribbean|   23|
|         Carnival|   22|
|         Princess|   17|
| Holland_American|   14|
|        Norwegian|   13|
|            Costa|   11|
|        Celebrity|   10|
|              MSC|    8|
|             Star|    6|
|              P&O|    6|
|Regent_Seven_Seas|    5|
|        Silversea|    4|
|         Seabourn|    3|
|         Windstar|    3|
|          Oceania|    3|
|           Cunard|    3|
|          Crystal|    2|
|          Azamara|    2|
|           Disney|    2|
|           Orient|    1|
+-----------------+-----+



In [115]:
df.createOrReplaceTempView('df')

In [127]:
df[df['Ship_name'] == 'Spirit'].select('crew').show()

+-----+
| crew|
+-----+
|10.29|
| 13.0|
|  1.6|
| 0.88|
+-----+



In [129]:
df[df['Cruise_line'] == 'Royal_Caribbean'].select('crew').show()

+-----+
| crew|
+-----+
|11.85|
| 8.48|
| 6.71|
|  7.6|
|11.76|
| 13.6|
|  7.6|
| 13.6|
| 8.69|
|  7.2|
| 13.6|
| 8.22|
|11.85|
| 8.22|
|11.85|
| 21.0|
| 8.68|
| 7.65|
| 8.58|
| 8.08|
+-----+
only showing top 20 rows



In [130]:
#spark.sql('SELECT * FROM df WHERE Ship_name = Spirit').show()

In [ ]:
df.orderBy()

# Preprocessing

1. String -> numeric column (stringindexer)
2. Feature scaling
3. vector assembling

In [24]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [42]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [34]:
strings = df.select(['Ship_name', 'Cruise_line'])

In [40]:
strings.columns

['Ship_name', 'Cruise_line']

In [43]:
indexers = [ StringIndexer(
    inputCol=column, outputCol = column+'_index'
) for column in strings.columns ]
pipeline = Pipeline(stages = indexers)

df_ix = pipeline.fit(df).transform(df)

In [45]:
df_ix.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- Ship_name_index: double (nullable = false)
 |-- Cruise_line_index: double (nullable = false)



In [61]:
list(set(df.columns) - set('crew'))

['crew',
 'Age',
 'cabins',
 'Cruise_line',
 'length',
 'passenger_density',
 'Ship_name',
 'Tonnage',
 'passengers']

In [46]:
from pyspark.ml.feature import StandardScaler

In [50]:
from pyspark.sql.functions import format_number

In [ ]:
df_ix = 

In [54]:
numeric = df.select(['Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density']).columns

In [55]:
scaler = [StandardScaler(
    inputCol=column, outputCol = column+'_scaled'
) for column in numeric ]

In [57]:
#pipeline_scaler = Pipeline(stages = scaler)
#df_final = pipeline_scaler.fit(df_ix).transform(df_ix)

In [58]:
## vector assembling

In [131]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [132]:
df_ix.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Ship_name_index',
 'Cruise_line_index']

In [151]:
assembler = VectorAssembler(inputCols = ['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'Cruise_line_index'], outputCol = 'features' )

In [152]:
output = assembler.transform(df_ix)

In [153]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,30.276999999...|
|[26.0,47.262,14.8...|
|[11.0,110.0,29.74...|
|[17.0,101.353,26....|
|[22.0,70.367,20.5...|
|[15.0,70.367,20.5...|
|[23.0,70.367,20.5...|
|[19.0,70.367,20.5...|
|[6.0,110.23899999...|
|[10.0,110.0,29.74...|
|[28.0,46.052,14.5...|
|[18.0,70.367,20.5...|
|[17.0,70.367,20.5...|
|[11.0,86.0,21.24,...|
|[8.0,110.0,29.74,...|
|[9.0,88.5,21.24,9...|
|[15.0,70.367,20.5...|
|[12.0,88.5,21.24,...|
|[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows



## Model train

In [154]:
FinalData = output.select(['features', 'crew'])

In [155]:
TrainData, TestData = FinalData.randomSplit([0.7, 0.3])

In [138]:
#TrainData.describe().show()

In [139]:
#TestData.describe().show()

In [156]:
lr = LinearRegression(labelCol='crew')

In [157]:
model = lr.fit(TrainData)

In [158]:
print('Coef: {}, intercept: {}'.format(model.coefficients, model.intercept))

Coef: [-0.015875687692805673,0.00408800509518473,-0.10464638193995804,0.33581189673653583,0.8422429295821642,-0.001384103891628938,0.05866401690668441], intercept: -0.6866653058133295


In [159]:
evaluate = model.evaluate(TestData)

In [160]:
evaluate.rootMeanSquaredError

0.9602579805252782

In [161]:
evaluate.r2

0.9077216066990401

In [162]:
evaluate.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -1.2541106363446843|
| -0.1729195918899027|
| -1.4620277791239058|
| -0.9298983548330328|
| -1.2146553962149387|
| -0.8791202495161539|
| -0.5791458563365381|
| -1.3577677848023058|
| -0.8052040489871253|
| -0.5196146220679818|
|  0.9208460261859859|
|  0.4380001144660266|
| -0.6364983003013194|
| -0.1964404590035258|
| -0.2524909801943984|
| -0.4488160245141426|
| -0.3498659690071797|
|-0.06667221130045142|
|  1.3458350359349627|
|  0.9847079792337308|
+--------------------+
only showing top 20 rows



In [163]:
FinalData.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [164]:
PredData = TestData.select('features')

In [165]:
Prediction = model.transform(PredData)

In [166]:
Prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,86.0,21.04,9...| 9.254110636344684|
|[5.0,115.0,35.74,...|12.372919591889902|
|[5.0,160.0,36.34,...|15.062027779123905|
|[6.0,30.276999999...| 4.479898354833033|
|[6.0,90.0,20.0,9....|10.214655396214939|
|[6.0,112.0,38.0,9...|11.779120249516154|
|[6.0,158.0,43.7,1...|14.179145856336538|
|[7.0,89.6,25.5,9....|11.227767784802305|
|[7.0,116.0,31.0,9...|12.805204048987125|
|[7.0,158.0,43.7,1...|14.119614622067981|
|[8.0,91.0,22.44,9...|10.079153973814014|
|[9.0,81.0,21.44,9...| 9.561999885533973|
|[9.0,110.0,29.74,...|12.236498300301319|
|[10.0,68.0,10.8,7...| 6.556440459003526|
|[10.0,81.76899999...| 8.672490980194398|
|[10.0,86.0,21.14,...| 9.648816024514142|
|[10.0,90.09,25.01...|  8.92986596900718|
|[10.0,91.62700000...| 9.066672211300451|
|[10.0,151.4,26.2,...|11.184164964065037|
|[11.0,90.0,22.4,9...| 10.01529202076627|
+--------------------+------------

If r2 is really high, it's always a good idea to do a reality check: whether 2 or more columns have high correlation

In [167]:
from pyspark.sql.functions import corr

In [169]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [173]:
corr = [df.select(corr('crew', column)).collect() for column in df.columns]

In [178]:
corr

[[Row(corr(crew, Ship_name)=nan)],
 [Row(corr(crew, Cruise_line)=None)],
 [Row(corr(crew, Age)=-0.5306565039638852)],
 [Row(corr(crew, Tonnage)=0.927568811544939)],
 [Row(corr(crew, passengers)=0.9152341306065384)],
 [Row(corr(crew, length)=0.8958566271016579)],
 [Row(corr(crew, cabins)=0.9508226063578497)],
 [Row(corr(crew, passenger_density)=-0.15550928421699717)],
 [Row(corr(crew, crew)=1.0)]]

In [ ]:
# This is the reason why our model has well performed. 